# Capstone Project - Data Wrangling

**PART 1 - Capstone Project for Springboard Intermediate Data Science: Python**

Prepared by: Ruhama Ahale (ruhama.ahale@gmail.com)

Mentored by: Raghunandan Patthar

October 2017

## **INTRODUCTION**

The client, an European Football Club wants a model to predict the aggregated overall rating of a player at each age of the player. They want to identify the current top players for each playing position and the significant attributes that affect the player rating for each position.


**DATA**: 

We will prepare the data for modeling, data is available in seperate tables such as **players** (player id, birthday, weight), **player_attributes** (finishing, diving, balance etc.) and **matches** (player coordinates for a match played). We join these tables to get a dataset with all required player details, attributes and favoured playing position. For each player we then aggregate data for each age of the player such that we have average overall rating, average finishing, etc for each age. This is the final dataset for analysis and model building, to fit a regression model to predict the overall rating we analyze the data to test assumptions and identify patterns

## **Importing the packages**

In [3]:
import os
import pandas as pd
import sqlite3
import warnings
import math
import numpy as np

warnings.filterwarnings("ignore")

## **Data Retrieving**

In [15]:
#connect to sqlite database and read files
countries = ""
matches = ""
leagues = ""
teams = ""

with sqlite3.connect('./database.sqlite') as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    players = pd.read_sql_query("SELECT * from Player", con)

len(players)

11060

## **Calculate coordinates for each players preferred position**

In [16]:
from scipy import stats

columns = ['player_api_id', 'player_name', 'matches_played', 'matches_played_in_position', 'preferred_x_position', 'preferred_y_position']
player_positions = pd.DataFrame([[0, 0, 0, 0, 0, 0]], columns=columns)     
cursor = con.cursor()

print("--------------------------------------------------------------------------------------------")
print("Calculations Started")
print("--------------------------------------------------------------------------------------------")

for index, player in players.iterrows():
    player_api_id = player['player_api_id']
    player_name = player['player_name']
    matches_played = 0
    print(str(index + 1) + ". Calculating positions for " + player_name)
    print("--------------------------------------------------------------------------------------------")
    x_positions_array = []
    y_positions_array = []
    for player_number in range(1,12):
        # Fetch x coordinates where player was home player
        x_positions = cursor.execute("""SELECT home_player_X%d FROM Match WHERE home_player_%d = '%d'""" % (player_number, player_number, player_api_id)).fetchall()
        if len(x_positions) > 0:
            matches_played += len(x_positions)
            x_positions = [x_position[0] for x_position in x_positions]
            x_positions_array += x_positions
        # Fetch x coordinates where player was away player
        x_positions = cursor.execute("""SELECT away_player_X%d FROM Match WHERE away_player_%d = '%d'""" % (player_number, player_number, player_api_id)).fetchall()
        if len(x_positions) > 0:
            matches_played += len(x_positions)
            x_positions = [x_position[0] for x_position in x_positions]
            x_positions_array += x_positions
        
        # Fetch y coordinates where player was home player
        y_positions = cursor.execute("""SELECT home_player_Y%d FROM Match WHERE home_player_%d = '%d'""" % (player_number, player_number, player_api_id)).fetchall()
        if len(y_positions) > 0:
            y_positions = [y_position[0] for y_position in y_positions]
            y_positions_array += y_positions
        # Fetch y coordinates where player was away player
        y_positions = cursor.execute("""SELECT away_player_Y%d FROM Match WHERE away_player_%d = '%d'""" % (player_number, player_number, player_api_id)).fetchall()
        if len(y_positions) > 0:
            y_positions = [y_position[0] for y_position in y_positions]
            y_positions_array += y_positions

    # Filter None values
    x_positions_array = list(filter(None, x_positions_array))
    y_positions_array = list(filter(None, y_positions_array))
    
    # Use 'mode' to get the preferred x and y coordinates
    # from all the coordinates that the player has played in
    preferred_x_position = stats.mode(x_positions_array)[0][0]
    preferred_y_position = stats.mode(y_positions_array)[0][0]
    matches_played_in_position = max(stats.mode(x_positions_array)[1][0], stats.mode(y_positions_array)[1][0])
    player_data = [player_api_id, player_name, matches_played, matches_played_in_position, preferred_x_position, preferred_y_position]
    player_data_frame = pd.DataFrame([player_data], columns=columns)
    player_positions = player_positions.append(player_data_frame, ignore_index=True)

print("Calculations Completed")
print("--------------------------------------------------------------------------------------------")

player_positions = player_positions.drop(player_positions.index[0])
player_positions.head()

--------------------------------------------------------------------------------------------
Calculations Started
--------------------------------------------------------------------------------------------
1. Calculating positions for Aaron Appindangoye
--------------------------------------------------------------------------------------------
2. Calculating positions for Aaron Cresswell
--------------------------------------------------------------------------------------------
3. Calculating positions for Aaron Doran
--------------------------------------------------------------------------------------------
4. Calculating positions for Aaron Galindo
--------------------------------------------------------------------------------------------
5. Calculating positions for Aaron Hughes
--------------------------------------------------------------------------------------------
Calculations Completed
--------------------------------------------------------------------------------------

,player_api_id,player_name,matches_played,matches_played_in_position,preferred_x_position,preferred_y_position
1,505942,Aaron Appindangoye,8,8,6,3
2,155782,Aaron Cresswell,75,75,8,3
3,162549,Aaron Doran,104,78,2,7
4,30572,Aaron Galindo,10,10,6,3
5,23780,Aaron Hughes,162,162,4,3


## **Decide player position based on the preffered position coordinates**

In [20]:
def playerPositionForPrefferedPositions(preferred_x_position, preferred_y_position):
    if (preferred_y_position == 1):
        return 'Goalkeeper'
    if (preferred_y_position > 1 and preferred_y_position <= 3):
        return 'Defender'
    if (preferred_y_position > 3 and preferred_y_position <= 8):
        return 'Midfielder'
    if (preferred_y_position > 8):
        return 'Forward'
    return 'Position Unknown'

for index, player in player_positions.iterrows():
    preferred_x_position = player['preferred_x_position']
    preferred_y_position = player['preferred_y_position']
    player_position = playerPositionForPrefferedPositions(preferred_x_position, preferred_y_position)
    player_positions.set_value(index, 'player_position', player_position)
    
player_positions.head()

,player_api_id,player_name,matches_played,matches_played_in_position,preferred_x_position,preferred_y_position,player_position
1,505942,Aaron Appindangoye,8,8,6,3,Defender
2,155782,Aaron Cresswell,75,75,8,3,Defender
3,162549,Aaron Doran,104,78,2,7,Midfielder
4,30572,Aaron Galindo,10,10,6,3,Defender
5,23780,Aaron Hughes,162,162,4,3,Defender


## **Save data to new table**

In [21]:
connection = sqlite3.connect("./database.sqlite")
player_positions.to_sql('Player_Positions', connection, if_exists = 'replace', index = False)

## **Get the league id of the latest match that a player played **

In [22]:
# Set a placeholder league id for the players
place_holder_league_id = 77777777777
players['league_id'] = place_holder_league_id

# Sort the matches by descending order of date
sorted_matches = matches.sort(['date'], ascending=[0])

In [25]:
# Get the list of players that player in a particular match
def playersFromMatch(match):
    match_players = []
    for player_number in range(1,12):
        home_player_key = 'home_player_%d' % player_number
        match_player = match[home_player_key]
        if not math.isnan(match_player):
            match_players.insert(0, str(int(match_player)))
        away_player_key = 'away_player_%d' % player_number
        match_player = match[away_player_key]
        if not math.isnan(match_player):
            match_players.insert(0, str(int(match_player)))
    return match_players

match_count = 0
# Iterate the matches in descending order and get the league id 
# for the latest match played by a player
for index, match in sorted_matches.iterrows():
    match_count = match_count + 1
    players_in_match = playersFromMatch(match)
    for index, player in players.iterrows():
        # If player played in a match, use the league id for that match
        if str(player['player_api_id']) in players_in_match:
            # Set teh player league id only if the player does not already have a league id
            if player['league_id'] == place_holder_league_id:
                players.set_value(index, 'league_id', match['league_id'])
    print("Calculating for match number %d" % match_count)
    players_without_league = players.loc[players['league_id'] == place_holder_league_id]
    print("%d players still misssing league id" % len(players_without_league))
    # Stop when all players have a league id
    if(len(players_without_league) <= 0):
        break
        
# These are the players which now have league id
players_with_league = players.loc[players['league_id'] != place_holder_league_id]
players_with_league.head()

Calculating for match number 1
10895 players still misssing league id
Calculating for match number 2
10895 players still misssing league id
Calculating for match number 3
10895 players still misssing league id
Calculating for match number 4
10895 players still misssing league id
Calculating for match number 5
10895 players still misssing league id


,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,league_id
102,103,38617,Adam Federici,164853,1985-01-31 00:00:00,187.96,198,1729
137,138,181211,Adama Traore,193177,1990-02-03 00:00:00,170.18,161,24558
245,246,34268,Alain Nef,49939,1982-02-06 00:00:00,190.50,194,24558
250,251,95220,Alain Wiss,184261,1990-08-21 00:00:00,180.34,148,24558
273,274,177689,Alban Pnishi,229013,1990-10-20 00:00:00,187.96,179,24558


## **Get the players current league and league's country **

In [26]:
# Set placeholder country and league names
players_with_league['league_name'] = "league name"
players_with_league['country_name'] = "country name"

for index, player in players_with_league.iterrows():
    player_league_id = player['league_id']
    # Get country name for player's league
    player_country = countries.loc[countries['id'] == player_league_id]['name'].iloc[0]
    players_with_league.set_value(index, 'country_name', player_country)
    # Get name for player's league
    player_league = leagues.loc[leagues['id'] == player_league_id]['name'].iloc[0]
    players_with_league.set_value(index, 'league_name', player_league)
    
players_with_league.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,league_id,league_name,country_name
102,103,38617,Adam Federici,164853,1985-01-31 00:00:00,187.96,198,1729,England Premier League,England
137,138,181211,Adama Traore,193177,1990-02-03 00:00:00,170.18,161,24558,Switzerland Super League,Switzerland
245,246,34268,Alain Nef,49939,1982-02-06 00:00:00,190.50,194,24558,Switzerland Super League,Switzerland
250,251,95220,Alain Wiss,184261,1990-08-21 00:00:00,180.34,148,24558,Switzerland Super League,Switzerland
273,274,177689,Alban Pnishi,229013,1990-10-20 00:00:00,187.96,179,24558,Switzerland Super League,Switzerland


## **Save data to new table**

In [27]:
connection = sqlite3.connect("./database.sqlite")
players_with_league.to_sql('Players_with_League_Country', connection, if_exists = 'replace', index = False)